# Analyze and evaluate optimization output

In [ ]:
import pickle
import pandas as pd
import seaborn as sns
import sys
from scipy.spatial import distance

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import matplotlib.pyplot as plt
import MEAutility as mu
import json
import time
import numpy as np
import LFPy
from pathlib import Path

import l5pc_model
import l5pc_evaluator
import l5pc_plot

%matplotlib notebook

In [ ]:
# Helper function to turn feature dicitonary into a list
def vectorize_features(feature_list):
    feature_vectors = []
    for feature in feature_list:
        feature_vector = {}
        for prot, prot_dict in feature.items():
            for loc, loc_feat in prot_dict.items():
                for feat, feat_val in loc_feat.items():
                    feature_vector[f'{prot}.{loc}.{feat}'] = feat_val[0]
        feature_vectors.append(feature_vector)
    return feature_vectors

In [ ]:
def plot_multiple_responses(responses_list, colors=None, return_fig=False):
    responses = responses_list[0]
    resp_no_mea = []
    for (resp_name, response) in sorted(responses.items()):
        if 'MEA' not in resp_name:
            resp_no_mea.append(resp_name)
    fig, axes = plt.subplots(len(resp_no_mea), figsize=(10, 10))
    for i, responses in enumerate(responses_list):
        if colors is None:
            color = f'C{i}'
        else:
            color = colors[i]
        for index, resp_name in enumerate(sorted(resp_no_mea)):
            response = responses[resp_name]
            axes[index].plot(response['time'], response['voltage'], label=resp_name, color=color)
            axes[index].set_title(resp_name)
    fig.tight_layout()
    fig.show()

    if return_fig:
        return fig

### Load gt params and optimization output

In [ ]:
result_folder = Path('tanguy_data/')
config_folder = Path('config')

In [ ]:
random_params = pd.read_csv(config_folder / 'params' / 'smart_random.csv', index_col='index')
random_params

In [ ]:
data = pickle.load(open(result_folder / 'runs_new.pkl', 'rb'))
df = pd.DataFrame(data)

In [ ]:
df.iloc[0].params_name

### Compute complete set of features for all samples

soma + bap + extra

In [ ]:
mea_positions = np.zeros((20, 3))
mea_positions[:, 2] = 20
mea_positions[:, 1] = np.linspace(-500, 1000, 20)
probe = mu.return_mea(info={'pos': list([list(p) for p in mea_positions]), 'center': False, 'plane': 'xy'})
electrode = LFPy.RecExtElectrode(probe=probe, method='linesource')

In [ ]:
channels = None

morphology = ephys.morphologies.NrnFileMorphology('morphology/C060114A7.asc', do_replace_axon=True)
param_configs = json.load(open('config/parameters.json'))
parameters = l5pc_model.define_parameters()
mechanisms = l5pc_model.define_mechanisms()

l5pc_cell = ephys.models.LFPyCellModel('l5pc', 
                                       v_init=-65., 
                                       morph=morphology, 
                                       mechs=mechanisms, 
                                       params=parameters)

param_names = [param.name for param in l5pc_cell.params.values() if not param.frozen]      
feature_set = 'all'

print(f'Feature set {feature_set}')
gt_responses = []

if feature_set in ["extra", "all"]:
    fitness_protocols = l5pc_evaluator.define_protocols(electrode) 
else:
    fitness_protocols = l5pc_evaluator.define_protocols() 

if feature_set in ["extra", "all"]:
    sim = ephys.simulators.LFPySimulator(LFPyCellModel=l5pc_cell, cvode_active=True, electrode=electrode)
else:
    sim = ephys.simulators.LFPySimulator(LFPyCellModel=l5pc_cell, cvode_active=True)

In [ ]:
if (result_folder / 'feats_responses.pkl').is_file():
    feats_responses = pickle.load((result_folder / 'feats_responses.pkl').open('rb'))
    feats = feats_responses['feats']
    responses = feats_responses['responses']
    gt_features_v = feats['gt']
    gt_responses = responses['gt']
    fitted_features_v = feats['fitted']
    fitted_responses = responses['fitted']
    compute_feats_responses = False
else:
    compute_feats_responses = True

In [ ]:
# Compute GT features and responses
if compute_feats_responses:
    gt_features = []
    gt_responses = []
    for i, (index, params) in enumerate(random_params.iterrows()):
        print(f'{i+1} / {len(random_params)}, {index}')

        feature_folder = f'config/features/{index}'
        _, response, feature_dict = l5pc_evaluator.compute_feature_values(params, l5pc_cell, fitness_protocols, sim, 
                                                                          feature_set=feature_set, probe=probe, 
                                                                          channels=channels,
                                                                          feature_folder=feature_folder,
                                                                          save_to_file=False)
        gt_features.append(feature_dict)
        gt_responses.append(response)
    gt_features_v = vectorize_features(gt_features)
else:
    print("Using loaded GT features and responses")

In [ ]:
# Compute fitted features and responses
if compute_feats_responses:
    fitted_features = []
    fitted_responses = []
    for i, (index, fit) in enumerate(df.iterrows()):
        params = pd.Series(data=fit['best_params'], index=fit['params_name'])
        print(f'{i+1} / {len(df)}')

        feature_folder = f'config/features/{index}'
        _, response, feature_dict = l5pc_evaluator.compute_feature_values(params, l5pc_cell, fitness_protocols, sim, 
                                                                          feature_set=feature_set, probe=probe, 
                                                                          channels=channels,
                                                                          feature_folder=feature_folder,
                                                                          save_to_file=False)
        fitted_features.append(feature_dict)
        fitted_responses.append(response)
    fitted_features_v = vectorize_features(fitted_features)
else:
    print("Using loaded FITTED features and responses")

In [ ]:
plot_multiple_responses(gt_responses)

In [ ]:
# Save features
save_features = True
if save_features and compute_feats_responses:
    feats = {'gt': gt_features_v, 'fitted': fitted_features_v}
    responses = {'gt': gt_responses, 'fitted': fitted_responses}
    dump_dict = {'feats': feats, 'responses': responses}
    with (result_folder / 'feats_responses.pkl').open('wb') as f:
        pickle.dump(dump_dict, f)
else:
    print("Responses and features already saved!")

In [ ]:
# Double check all GT params produce responses with all BAP features (5)
complete_bap = []
for i, gt in enumerate(list(gt_features_v)):
    bap_feat = [k for k in gt.keys() if 'bAP' in k]
    if len(bap_feat) == 5:
        complete_bap.append(i)
print(len(complete_bap))

### Compute distances

In [ ]:
param_distances = []
param_distances_apical = []
param_distances_somatic = []
param_distances_axonal = []

feature_distances = []
feat_soma_dist = []
feat_dend_dist = []
feat_mea_dist = []

# channels = [4,5,6,10,15]
channels=None
for i, (index, fit) in enumerate(df.iterrows()):
    sample_id = int(fit.sample_id)
    gt_params = random_params.iloc[sample_id].sort_index()
    fit_params = pd.Series(fit.best_params, fit.params_name).sort_index()
    
    axonal_idxs = gt_params.index.str.contains('axonal')
    somatic_idxs = gt_params.index.str.contains('somatic')
    apical_idxs = gt_params.index.str.contains('apical')
    
    param_dist = distance.cosine(fit_params.values, gt_params.values)
    param_dist_ax = distance.cosine(fit_params[axonal_idxs].values, gt_params[axonal_idxs].values)
    param_dist_som = distance.cosine(fit_params[somatic_idxs].values, gt_params[somatic_idxs].values)
    param_dist_ap = distance.cosine(fit_params[apical_idxs].values, gt_params[apical_idxs].values)
    
    param_distances.append(param_dist)
    param_distances_axonal.append(param_dist_ax)
    param_distances_somatic.append(param_dist_som)
    param_distances_apical.append(param_dist_ap)
    
    
    selected_keys = []
    for k in gt_features_v[sample_id].keys():
        if 'MEA' not in k:
            selected_keys.append(k)
        else:
            if channels is not None:
                if int(k[-1]) in channels:
                    selected_keys.append(k)
                else:
                    selected_keys.append(k)
    gt_feat, fitted_feat = [], []
    gt_feat_soma, gt_feat_dend, gt_feat_mea = [], [], []
    fitted_feat_soma, fitted_feat_dend, fitted_feat_mea = [], [], []
    for k in selected_keys:
        if k in gt_features_v[sample_id] and k in fitted_features_v[i]:
            gt_feat.append(gt_features_v[sample_id][k])
            fitted_feat.append(fitted_features_v[i][k])
            if 'soma' in k:
                gt_feat_soma.append(gt_features_v[sample_id][k])
                fitted_feat_soma.append(fitted_features_v[i][k])
            if 'dend' in k:
                gt_feat_dend.append(gt_features_v[sample_id][k])
                fitted_feat_dend.append(fitted_features_v[i][k])
            if 'mea' in k:
                gt_feat_mea.append(gt_features_v[sample_id][k])
                fitted_feat_mea.append(fitted_features_v[i][k])
                
    feature_dist = distance.cosine(fitted_feat, gt_feat)
    feature_dist_soma = distance.cosine(fitted_feat_soma, gt_feat_soma)
    feature_dist_dend = distance.cosine(fitted_feat_dend, gt_feat_dend)
    feature_dist_mea = distance.cosine(fitted_feat_mea, gt_feat_mea)
    
    feature_distances.append(feature_dist)
    feat_soma_dist.append(feature_dist_soma)    
    feat_dend_dist.append(feature_dist_dend)    
    feat_mea_dist.append(feature_dist_mea)    

In [ ]:
df['param_dist'] = param_distances
df['param_dist_apical'] = param_distances_apical
df['param_dist_axonal'] = param_distances_axonal
df['param_dist_somatic'] = param_distances_somatic

df['feat_dist'] = feature_distances
df['feat_dist_soma'] = feat_soma_dist
df['feat_dist_dend'] = feat_dend_dist
df['feat_dist_mea'] = feat_mea_dist

In [ ]:
fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
sns.barplot(data=df, x='feature_set', y='param_dist', hue='sample_id', ax=ax1, alpha=0.5)
ax1.set_title("All params")

In [ ]:
fig2 = plt.figure()
ax21 = fig2.add_subplot(131)
ax22 = fig2.add_subplot(132)
ax23 = fig2.add_subplot(133)
sns.barplot(data=df, x='feature_set', y='param_dist_somatic', hue='sample_id', ax=ax21, alpha=0.5, ci=None)
sns.barplot(data=df, x='feature_set', y='param_dist_axonal', hue='sample_id', ax=ax22, alpha=0.5, ci=None)
sns.barplot(data=df, x='feature_set', y='param_dist_apical', hue='sample_id', ax=ax23, alpha=0.5, ci=None)

ax21.set_title("Somatic params")
ax22.set_title("Axonal params")
ax23.set_title("Apical params")

In [ ]:
fig3 = plt.figure()
ax3 = fig3.add_subplot(111)
sns.boxenplot(data=df, x='feature_set', y='feat_dist', hue='sample_id', ax=ax3)
ax3.set_title('All features')

In [ ]:
fig4 = plt.figure()
ax41 = fig4.add_subplot(131)
ax42 = fig4.add_subplot(132)
ax43 = fig4.add_subplot(133)
sns.boxenplot(data=df, x='feature_set', y='feat_dist_soma', hue='sample_id', ax=ax41)#, alpha=0.5, ci=None)
sns.boxenplot(data=df, x='feature_set', y='feat_dist_dend', hue='sample_id', ax=ax42)#, alpha=0.5, ci=None)
sns.boxenplot(data=df, x='feature_set', y='feat_dist_mea', hue='sample_id', ax=ax43)#, alpha=0.5, ci=None)

ax41.set_title("Somatic features")
ax42.set_title("Dend features")
ax43.set_title("MEA features")
ax42.set_yticks([])
ax42.set_yticklabels([])
ax42.set_ylabel('')
ax43.set_yticks([])
ax43.set_yticklabels([])
ax43.set_ylabel('')

In [ ]:
gt_id = 0
df_fit = df[df.sample_id == str(gt_id)]
fitted = np.array(fitted_responses)[np.array(df_fit.index)]

In [ ]:
colors = {'soma': 'C0', 'bap': 'C1', 'extra': 'C2'}

In [ ]:
color_list = []
feature_sets = []
for i in range(len(df_1)):
    color_list.append(colors[df_fit.iloc[i].feature_set])
    feature_sets.append(df_fit.iloc[i].feature_set)

soma_idxs = np.where(np.array(feature_sets) == 'soma')
bap_idxs = np.where(np.array(feature_sets) == 'bap')
extra_idxs = np.where(np.array(feature_sets) == 'extra')

fitted_soma = fitted[soma_idxs]
fitted_bap = fitted[bap_idxs]
fitted_extra = fitted[extra_idxs]

color_list.append('k')

In [ ]:
fig_soma = plot_multiple_responses(responses_list=np.concatenate((fitted_soma, [gt_responses[gt_id]])), 
                                   colors=['C0'] * len(fitted_soma) + ['k'], return_fig=True)
fig_soma.suptitle("Soma features", fontsize=30, y=0.98)
fig_soma.subplots_adjust(top=0.9)

In [ ]:
fig_bap = plot_multiple_responses(responses_list=np.concatenate((fitted_bap, [gt_responses[gt_id]])), 
                                  colors=['C1'] * len(fitted_bap) + ['k'], return_fig=True)
fig_bap.suptitle("BAP features", fontsize=30, y=0.98)
fig_bap.subplots_adjust(top=0.9)

In [ ]:
fig_extra = plot_multiple_responses(responses_list=np.concatenate((fitted_extra, [gt_responses[gt_id]])), 
                                    colors=['C2'] * len(fitted_extra) + ['k'], return_fig=True)
fig_extra.suptitle("Extra features", fontsize=30, y=0.98)
fig_extra.subplots_adjust(top=0.9)

## Plot extracellular action potentials

In [ ]:
fig_eap_soma = plt.figure(figsize=(7, 12))
ax_eap_soma = fig_eap_soma.add_subplot(111)
for i, fitted in enumerate(fitted_soma):
    eap = l5pc_evaluator.calculate_eap(responses=fitted, protocols=fitness_protocols,
                                       protocol_name='Step1')
    eap_norm = eap / np.max(np.abs(eap), 1, keepdims=True)
    mu.plot_mea_recording(eap_norm, 
                          probe, colors='C0', ax=ax_eap_soma, vscale=2)
eap = l5pc_evaluator.calculate_eap(responses=gt_responses[gt_id], protocols=fitness_protocols,
                                   protocol_name='Step1')
eap_norm = eap / np.max(np.abs(eap), 1, keepdims=True)
mu.plot_mea_recording(eap_norm, 
                      probe, colors='k', ax=ax_eap_soma, vscale=2)
ax_eap_soma.set_title("Soma features", fontsize=30)

In [ ]:
fig_eap_bap = plt.figure(figsize=(7, 12))
ax_eap_bap = fig_eap_bap.add_subplot(111)
for i, fitted in enumerate(fitted_bap):
    eap = l5pc_evaluator.calculate_eap(responses=fitted, protocols=fitness_protocols,
                                       protocol_name='Step1')
    eap_norm = eap / np.max(np.abs(eap), 1, keepdims=True)
    mu.plot_mea_recording(eap_norm, 
                          probe, colors='C1', ax=ax_eap_bap, vscale=2)
eap = l5pc_evaluator.calculate_eap(responses=gt_responses[gt_id], protocols=fitness_protocols,
                                   protocol_name='Step1')
eap_norm = eap / np.max(np.abs(eap), 1, keepdims=True)
mu.plot_mea_recording(eap_norm, 
                      probe, colors='k', ax=ax_eap_bap, vscale=2)
ax_eap_bap.set_title("BAP features", fontsize=30)

In [ ]:
fig_eap_extra = plt.figure(figsize=(7, 12))
ax_eap_extra = fig_eap_extra.add_subplot(111)
for i, fitted in enumerate(fitted_extra):
    eap = l5pc_evaluator.calculate_eap(responses=fitted, protocols=fitness_protocols,
                                       protocol_name='Step1')
    eap_norm = eap / np.max(np.abs(eap), 1, keepdims=True)
    mu.plot_mea_recording(eap_norm, 
                          probe, colors='C2', ax=ax_eap_extra, vscale=2)
eap = l5pc_evaluator.calculate_eap(responses=gt_responses[gt_id], protocols=fitness_protocols,
                                   protocol_name='Step1')
eap_norm = eap / np.max(np.abs(eap), 1, keepdims=True)
mu.plot_mea_recording(eap_norm, 
                      probe, colors='k', ax=ax_eap_extra, vscale=2)
ax_eap_extra.set_title("Extra features", fontsize=30)